In [1]:
import krpc

In [2]:
conn = krpc.connect()
print(conn.krpc.get_status().version)

0.4.8


In [3]:
# get the vessel and print its altitude
vessel = conn.space_center.active_vessel

vessel.name = "My Vessel"

flight_info = vessel.flight()
print(flight_info.mean_altitude)

96.48136745078955


## data streaming from the server
A usecase for kRPC is to continuously extract data from the game. Naive approach such as RPC calls repeatedly requires significant communication overhead.  
__More efficient mechanism called streams can achieve this without overhead.__  
A stream repeatedly executes a procedure on the server(with a fixed set of argument values) and sends the result to the client. It only requires a single message to be sent to the server to establish the stream, __which will then continuously send data to the client until the stream is closed.__

# Set up stream for method calls and attributes
## 1. Set up stream for method calls
```
refframe = vessel.orbit.body.reference_frame
position = conn.add_stream(vessel.position, refframe)
while True:
    print(position())
```
A stream can be created for any procedure that returns a value. This includes both method calls and attribute accesses. The examples above 

## 2. set up stream for ATTRIBUTE ACCESSES
```
altitude = conn.add_stream(getattr, flight_info, 'mean_altitude')
for i in range(10):
    print(altitude())
```
## 3. remove stream
```
krpc.stream.Stream.remove(altitude)
```


In [14]:
# set up a stream for METHOD CALLS
refframe = vessel.orbit.body.reference_frame
position = conn.add_stream(vessel.position, refframe)
for i in range(10):
    print(position())

(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)
(159787.53743526462, -1014.1873623253449, -578431.2415962408)


A stream can be created for any procedure that returns a value. This includes both method calls and attribute accesses. The examples above 

In [11]:
# stream for ATTRIBUTE ACCESSES
altitude = conn.add_stream(getattr, flight_info, 'mean_altitude')
for i in range(10):
    print(altitude())

96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322
96.48136768839322


#### remove stream:

In [12]:
krpc.stream.Stream.remove(altitude)

# Synchronizing with stream updates
wait until the value returned by a method or attribute changes, and then take some action.
# two mechanisms to do this efficiently
### 1. condition variables
The condition variables are instances of `threading.Condition` from python standard library. They can be used to block the current thread of execution until the value of the stream changes.  
Example: waits until the `abort` button is pressed in game, or by waiting for the value of `vessel.control.abort` to change to true.
```
with conn.stream(getattr, vessel.control, 'abort') as abort:
    with abort.condition:
        while not abort():
            abort.wait()
```

### 2. callbacks
take single argument, which is the new value of the stream, and should return nothing.  
Following registers two callbacks that are invoked when the value of `vessel.control.abort` changes.
```
abort = conn.add_stream(getattr, vessel.control, 'abort')

def check_abort1(x):
    print("Abort 1 called with value of ", x)
    
def check_abort2(x):
    print("Abort 2 called with value of ", x)
    
abort.add_callback(check_abort1)
abort.add_callback(check_abort1)
abort.start()

# keep program running
while True:
    pass
```


In [4]:
help(conn.space_center)

Help on SpaceCenter in module krpc.service object:

class SpaceCenter(ServiceBase)
 |  Provides functionality to interact with Kerbal Space Program. This includes controlling
 |  the active vessel, managing its resources, planning maneuver nodes and auto-piloting.
 |  
 |  Method resolution order:
 |      SpaceCenter
 |      ServiceBase
 |      krpc.types.DynamicType
 |      builtins.object
 |  
 |  Static methods defined here:
 |  
 |  can_rails_warp_at(factor=1)
 |      Returns True if regular "on-rails" time warp can be used, at the specified warp
 |      factor. The maximum time warp rate is limited by various things,
 |      including how close the active vessel is to a planet. See
 |      the KSP wiki
 |      for details.
 |      
 |      Args:
 |          factor: The warp factor to check.
 |  
 |  clear_target()
 |      Clears the current target.
 |  
 |  launch_vessel(craft_directory, name, launch_site, recover=True)
 |      Launch a vessel.
 |      
 |      Args:
 |          c